# Longest increasing subsequence
1.  any letter can be chosen or not, so the total possibility is 2<sup>N</sup>. Brute force is not a good idea
2. to use dynamic programming, the key is to find an optimal value function, typically with the help of intermediate variables
3. for arr[0, 1, ...N-1], define L[i], so that L[i] is the lis of arr[0, 1, ...i]. Because arr[i] is the last item, the optimal value function is easy to be calculated. L[i] = max(L[j] + 1) with condition j < i, and arr[i] > arr[j]. 

Assume arr = [3, 1, 4, 2, 3], then
1. L[0] = 1, {3}
2. L[1] = 1, {1}
3. L[2] = 2, {1, 4}
4. L[3] = 2, {1, 2}
5. L[4] = 3, {1, 2, 3}

Details:
1. Line 1 and 2 is straightforward. Line 3: L[2]= l[0] + 1, then L[2] = L[1] + 1 since 4 is larger than 3 and 1. In other words it will be calculated twice but ended up with the same number
2. Line 4: since the only smaller number than 2 is 1, then L[3] = L[1] + 1
3. Line 5: 3 is larger than 1 and 2 so L[3] will be calculated twice. L[4] = L[2] + 1, then overwritten by L[4] = L[3] +1

Conclusions:
The above algorithm will be N<sup>2</sup> due to the inner loop. We could find new ways for binary search to get O(nlogn)

In [9]:
# O(N^2)
def lis(arr):
    length = len(arr)
    inter_arr = [1] * length
    for i in range(1, length):
        for j in range(0, i):
            if arr[i] > arr[j] and inter_arr[i] < inter_arr[j] + 1:
                inter_arr[i] = inter_arr[j] + 1
    return inter_arr

arr = [1, 3, 5, 6, 1, 4, 2, 3, 7, 8]
lis(arr)

[1, 2, 3, 4, 1, 3, 2, 3, 5, 6]

In order to reach O(nlogn), we will need to have a special array that
1) It holds some dynamic information which will be constantly updated
2) The array is sorted so that we can do binary search

Let M[j-1] to represent the position in arr, which is the smallest number of length j (or LIS)
Think about two example

seq = [10, 20, 30] ==> M = [0, 1, 2], P = [None, 0, 1]

seq = [10, 20, 30, 1, 2] ==>M = [3, 4, 2, None, None, P = [None, 0, 1, None, 3]

seq = [10, 20, 30, 1, 2, 5] ==>M = [3, 4, 5, None, None, None], P = [None, 0, 1, None, 3, 4]

seq = [10, 20, 30, 1, 2, 40] ==> M = [3, 4, 2, 5, None, None], P = [None, 0, 1, None, 3, 2]

So M will represent the current LIS, then will be replaced by a smaller number. 
1)  smaller number will not take effect unless the total length is larger than previous LIS. Until [1,2,5] the previous three numbers are fully replaced. If 40 is followed after [1, 2], then 40 will be counted together with [10, 20, 30]
2) The update part is a search of ordered tree, such as [10, 20, 30]. So binary search is possible.

In [62]:
def subsequence(seq):
    if not seq:
        return seq

    M = [None] * len(seq)    # offset by 1 (j -> j-1)
    P = [None] * len(seq)

    # Since we have at least one element in our list, we can start by 
    # knowing that the there's at least an increasing subsequence of length one:
    # the first element.
    L = 1
    M[0] = 0

    # Looping over the sequence starting from the second element
    for i in range(1, len(seq)):
        # Binary search: we want the largest j <= L
        #  such that seq[M[j]] < seq[i] (default j = 0),
        #  hence we want the lower bound at the end of the search process.
        lower = 0
        upper = L
        print("****starting", i, seq[i])

        # Since the binary search will not look at the upper bound value,
        # we'll have to check that manually
        if seq[M[upper-1]] < seq[i]:
            j = upper

        else:
            # actual binary search loop
            while upper - lower > 1:
                mid = (upper + lower) // 2
                if seq[M[mid-1]] < seq[i]:
                    lower = mid
                else:
                    upper = mid

            j = lower    # this will also set the default value to 0

        P[i] = M[j-1]

        if j == L or seq[i] < seq[M[j]]:
            M[j] = i
            L = max(L, j+1)
        
        print("M =", M)
        print("P =", P)
        print(L)

    # Building the result: [seq[M[L-1]], seq[P[M[L-1]]], seq[P[P[M[L-1]]]], ...]
    result = []
    pos = M[L-1]
    for _ in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return result[::-1]    # reversing

seq = [10, 20, 30]
subsequence(seq)

****starting 1 20
M = [0, 1, None]
P = [None, 0, None]
2
****starting 2 30
M = [0, 1, 2]
P = [None, 0, 1]
3


[10, 20, 30]